In [ ]:
from pathlib import Path
import sqlite3
import pandas as pd
from itertools import combinations

In [ ]:
BASE_DIR = Path().resolve()
sql_path = BASE_DIR / "Assignment__1.sql"
db_path  = BASE_DIR / "assignment1.db"

In [ ]:
# 1) Build (or rebuild) the database from the SQL script
with sqlite3.connect(db_path) as con:
    with open(sql_path, "r", encoding="utf-8") as f:
        sql_script = f.read()
    # Execute the schema & sample data; enable foreign keys
    con.executescript("PRAGMA foreign_keys=ON;\n" + sql_script)

In [ ]:
# 2) Quick sanity check
with sqlite3.connect(db_path) as con:
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table';", con
    )
    print("Tables in database:", tables)

In [ ]:
# Example: list tables
with sqlite3.connect(db_path) as con:
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table';", con
    )
tables

In [ ]:
query = """
SELECT c.customer_id,
       c.street || ' ' || c.house_number AS address,
       COALESCE(SUM(cp.contract_amount), 0) AS total_contract_amount
FROM customer AS c
LEFT JOIN customer_partner AS cp
       ON c.customer_id = cp.customer_id
GROUP BY c.customer_id
ORDER BY total_contract_amount DESC;
"""
with sqlite3.connect(db_path) as con:
    total_contracts = pd.read_sql_query(query, con)
total_contracts.head()


In [ ]:
# Task 3B – Read the customer table into a pandas DataFrame
with sqlite3.connect(db_path) as con:
    customer_df = pd.read_sql_query("SELECT * FROM customer;", con)

# Preview the first few rows
customer_df.head()


In [ ]:
customer_df.info()       # column types & non-null counts
customer_df.describe()   # numeric summary statistics
customer_df['city'].value_counts()


In [ ]:
# quick helper to clean up values so small differences (spaces, case) don't matter
def clean_val(x):
    if pd.isna(x):
        return ""
    return str(x).strip().lower()

# guess which column is the unique ID (so we can ignore it in the similarity check)
def find_id_col(df):
    if "customer_id" in df.columns:
        return "customer_id"
    for c in df.columns:
        name = c.lower()
        if ("customer" in name and "id" in name) or name == "id" or name.endswith("_id"):
            return c
    # fall back to first column if nothing obvious
    return df.columns[0]

# turn one row into a set of its attribute values (except the id)
def row_to_set(row, ignore_cols=()):
    return {clean_val(v) for k, v in row.items() if k not in ignore_cols}

# overlap coefficient = size of intersection / size of smaller set
def overlap(row1, row2, ignore_cols=()):
    set1 = row_to_set(row1, ignore_cols)
    set2 = row_to_set(row2, ignore_cols)
    if not set1 or not set2:
        return 0.0
    return len(set1 & set2) / min(len(set1), len(set2))

def compare_rows(df, cutoff=0.7):
    """
    Go through every pair of rows and compute the overlap coefficient.
    Keep only the pairs whose similarity is at least 'cutoff'.
    Returns a DataFrame with the IDs, similarity score and a few context columns.
    """
    id_col = find_id_col(df)
    hits = []

    # make a cleaned copy just for comparison
    df_clean = df.copy()
    for c in df_clean.columns:
        df_clean[c] = df_clean[c].apply(clean_val)

    for i, j in combinations(range(len(df_clean)), 2):
        score = overlap(df_clean.iloc[i], df_clean.iloc[j], ignore_cols=(id_col,))
        if score >= cutoff:
            match = {
                "id_1": df.iloc[i][id_col],
                "id_2": df.iloc[j][id_col],
                "similarity": round(score, 3),
            }
            # add a few descriptive columns if they exist
            for c in ("street", "house_number", "city", "country"):
                if c in df.columns:
                    match[f"{c}_1"] = df.iloc[i][c]
                    match[f"{c}_2"] = df.iloc[j][c]
            hits.append(match)

    if not hits:
        print(f"No pairs with similarity ≥ {cutoff}")
        return pd.DataFrame()  # return empty if nothing found

    # sort best matches first
    return pd.DataFrame(hits).sort_values("similarity", ascending=False).reset_index(drop=True)


In [ ]:
similar_pairs = compare_rows(customer_df, cutoff=0.7)
similar_pairs.head()